In [1168]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy import spatial

import matplotlib.pyplot as plt # side-stepping mpl backend

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_selection import SelectPercentile, f_classif
# from sklearn import cross_validation
import heapq
import string
import re

from nltk.corpus import stopwords
from nltk import word_tokenize
from collections import defaultdict
from collections import Counter

trainingData  = pd.read_csv('data/train.dat.txt', sep="\t", encoding='utf-8', header=None, names=["rating","review"])[:2];
testData = pd.read_csv('data/test.dat.txt', sep="\t", encoding='utf-8', header=None, names=["review"])[:2];


In [1169]:
# 
print('Before Cleaning');
print (trainingData[0:1])

def preProcess(reviews):
#     print(reviews);
    processedReviews = [];
    for review in reviews:
        tokens = word_tokenize(review);
        filteredTokens = [];
        for token in list(tokens):
#             print(token);
            # if it is a stopword then eliminate
            if token.lower() in stopwords.words('english'):
#                 print('removing stopword ' + token);
                continue;
            # if it is punctuation then eliminate
            if token.lower() in set(string.punctuation):
#                 print('removing punct ' + token);
                continue;
            if len(token)<=3:
#                 print('removing small ' + token);
                continue;
            token = token.lower();
            filteredTokens.append(token);
#         print(filteredTokens);
        processedReviews.append(filteredTokens);
#     print(len(processedReviews));
    return processedReviews;
print('After Cleaning')
XTrain = preProcess(trainingData['review']);
print(XTrain);
print (len(XTrain));
XTest = preProcess(testData['review']);
print (len(XTest));

Before Cleaning
   rating                                             review
0      -1  Although a film with Bruce Willis is always wo...
After Cleaning
[[u'although', u'film', u'bruce', u'willis', u'always', u'worth', u'watching', u'better', u'skip', u'watched', u'television', u'plunk', u'cash', u'lucky', u'plot', u'develops', u'slowly', u'slowly', u'although', u'first', u'minutes', u'quite', u'believable', u'gets', u'unbelievable', u'towards', u'highly', u'questionable', u'seasoned', u'soldier', u'like', u'waters', u'would', u'disobey', u'direct', u'orders', u'even', u'would', u'rest', u'platoon', u'would', u'know', u'puts', u'direct', u'danger', u'know', u'certainly', u'follow', u'heck', u'says', u'despite', u'direct', u'orders', u'remember', u'still', u'nice', u'scenes', u'movie', u'somewhat', u'save', u'village', u'total', u'population', u'massacred', u'rebels', u'well', u'save', u'dozen', u'villagers', u'rest', u'already', u'killed', u'strange', u'part', u'take', u'trucks', u'reb

In [1170]:
# count frequencies for all words in the Training data
def countFrequency(data):
    wordCountInData = Counter()
    for d in data:
    #     print(d);
        for w in d:
    #         print(w);
            if w not in wordCountInData:
                wordCountInData[w]=1
            else:
                wordCountInData[w] += 1
    print("Number of unique words: %d." % len(wordCountInData));
    return wordCountInData;

wordCountInTrainingData = countFrequency(XTrain);
wordCountInTestData = countFrequency(XTest);
print(len(wordCountInTrainingData));
print(len(wordCountInTestData));

Number of unique words: 191.
Number of unique words: 152.
191
152


In [1171]:
#Top percent of training data
topPct = 1;
topPctOfTraining = wordCountInTrainingData.most_common(int(round(len(wordCountInTrainingData)*topPct)));
topPctOfTest = wordCountInTestData.most_common(int(round(len(wordCountInTestData)*topPct)));

# Get a set of topPctOfTraining
topPctOfTrainingSet = set()
for word in topPctOfTraining:
    topPctOfTrainingSet.add(word[0]);

# Get a set of topPctOfTest
topPctOfTestSet = set()
for word in topPctOfTest:
    topPctOfTestSet.add(word[0]);
    
# print(topPctOfTrainingSet);
# print(topPctOfTestSet);
print(len(topPctOfTrainingSet));
print(len(topPctOfTestSet));

191
152


In [1172]:
testTrainSet =  set.intersection(topPctOfTrainingSet,topPctOfTestSet);
print("Number of unique words common in both test and train: %d." % len(testTrainSet));
# print(testTrainSet);

Number of unique words common in both test and train: 17.


In [1173]:
cnt = Counter(testTrainSet);
top_percentile = 1.0
features = cnt.most_common(int(round(len(cnt)*top_percentile)))
featuresCount = len(features)
# print(features);
print(featuresCount)

17


In [1174]:
featureList= [];
for feature in features:
    featureList.append(feature[0])
print(len(featureList));
print(featureList);

17
[u'even', u'made', u'point', u'movie', u'loose', u'scenes', u'first', u'times', u'every', u'take', u'characters', u'time', u'nothing', u'still', u'think', u'film', u'nice']


In [1175]:
# #Buid CSR matrix

def build_matrix1(mat):
    scipy.sparse.csr_matrix(mat)

from collections import Counter
from scipy.sparse import csr_matrix
def build_matrix(docs):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    dim = len(featureList)
    feature_set = set(featureList[:dim])
    nrows = len(docs)
    idx = {}
    tid = 0
    nnz = 0
    for d in docs:
#         print('d---')
#         print(d)
#         set_d = set(d)
        set_d = set()
        for word in d:
#             print(word)
            set_d.add(word);
        print('set d--')
        print(set_d);
        print('features--')
        print(feature_set);
        d = list(set.intersection(feature_set,set_d))
        print('common elemets %d',len(d));
        print('com')
        nnz += len(set(d))
        for w in d:
            print('adding word ', w) ;
            if w not in idx:
                idx[w] = tid
                tid += 1
    ncols = len(idx)
    print(idx);
    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=np.int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        set_d = set(d)
        d = list(set.intersection(feature_set,set_d))
        cnt = Counter(d)
        keys = list(k for k,_ in cnt.most_common())
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat


def csr_info(mat, name="", non_empy=False):
    r""" Print out info about this CSR matrix. If non_empy, 
    report number of non-empty rows and cols as well
    """
    if non_empy:
        print("%s [nrows %d (%d non-empty), ncols %d (%d non-empty), nnz %d]" % (
                name, mat.shape[0], 
                sum(1 if mat.indptr[i+1] > mat.indptr[i] else 0 
                for i in range(mat.shape[0])), 
                mat.shape[1], len(np.unique(mat.indices)), 
                len(mat.data)))
    else:
        print( "%s [nrows %d, ncols %d, nnz %d]" % (name, 
                mat.shape[0], mat.shape[1], len(mat.data)) )

In [1176]:
docsTrainData = [word_tokenize(l) for l in trainingData['review']]
matTrainData  = build_matrix(docsTrainData);
csr_info(matTrainData);

docsTestData = [word_tokenize(l) for l in testData['review']]
matTestData  = build_matrix(docsTestData)
csr_info(matTestData)

set d--
set([u'rating', u'What', u'Who', u'rebels', u'being', u'skip', u'laptop', u'saves', u'Lucky', u'rest', u'gun', u'brain', u'kill', u'go', u'follow', u'bunch', u'certainly', u'group', u'earned', u'(', u'point', u'claymore', u',', u'killed', u'better', u'to', u'only', u'Carrying', u'Although', u'save', u'villagers', u'worth', u'gave', u'it.', u'do', u'them', u'his', u'which', u'watching', u'very', u'know', u'they', u'despite', u'foot', u'highly', u'Where', u'him', u'nearly', u'enemy', u'like', u'trucks', u'did', u'die', u'Anyway', u'platoon', u'always', u'where', u'round', u'population', u'because', u'says', u'me.', u'some', u'direct', u'around', u'scenes', u'insult', u'Only', u'are', u'soldiers', u'our', u'Lt.', u'out', u'even', u"n't", u'what', u'give', u'for', u'plunk', u'Bruce', u'movie', u'mines', u'/', u'your', u'behind', u'stuff', u'run', u'7', u'got', u'get', u'?', u'shows', u'fight', u'dozen', u'loose', u'quite', u'completely', u'not', u'let', u'br', u'hundreds', u'by', u

In [1177]:
# scale and normalize the matrix
def csr_idf(mat, copy=False, **kargs):
    r""" Scale a CSR matrix by idf. 
    Returns scaling factors as dict. If copy is True, 
    returns scaled matrix and scaling factors.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # document frequency
    df = defaultdict(int)
    for i in ind:
        df[i] += 1
    # inverse document frequency
    for k,v in df.items():
        df[k] = np.log(nrows / float(v))  ## df turns to idf - reusing memory
    # scale by idf
    for i in range(0, nnz):
        val[i] *= df[ind[i]]
        
    return df if copy is False else mat

def csr_l2normalize(mat, copy=False, **kargs):
    r""" Normalize the rows of a CSR matrix by their L-2 norm. 
    If copy is True, returns a copy of the normalized matrix.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = 1.0/np.sqrt(rsum)
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
            
    if copy is True:
        return mat

In [1178]:
scaledTrainData = csr_idf(matTrainData, copy=True)
normalizedTrainData = csr_l2normalize(scaledTrainData, copy=True)
# print("csr matrix:", matTrainData.todense(), "\n")
# print("scaled csr matrix:", scaledTrainData.todense(), "\n")
# print("normalized csr matrix:", normalizedTrainData.todense())

In [1179]:
scaledTestData = csr_idf(matTestData, copy=True)
normalizedTestData = csr_l2normalize(scaledTestData, copy=True)
# print("csr matrix:", matTestData.todense(), "\n")
# print("scaled csr matrix:", scaledTestData.todense(), "\n")
# print("normalized csr matrix:", normalizedTestData.todense())

In [1180]:
# Calculate consine similarity between training and test normalized data
from sklearn.metrics.pairwise import cosine_similarity
def calculate_cosine_sim(train,test):
    cosineSimilarity = cosine_similarity(train,test)
    return cosineSimilarity
csr_info(normalizedTrainData)
csr_info(normalizedTestData)
cosineSimilarity = calculate_cosine_sim(normalizedTrainData, normalizedTestData);
print(len(cosineSimilarity));

 [nrows 2, ncols 16, nnz 20]
 [nrows 2, ncols 17, nnz 20]


ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 16 while Y.shape[1] == 17

In [ ]:
# Find K nearest neighbors and write to test file
f = open('data/test_out.dat.txt', 'w')
count = 0
for row in cosineSimilarity:
    k=1
    kLargestSimilarities = np.argpartition(-row, k)
    print(kLargestSimilarities)
    neighbors = kLargestSimilarities[:k]
    print(neighbors)
    neighbourReviewClassList = []
    neighbourReviewClassNegative = 0
    neighbourReviewClassPositive = 0

    for review in neighbors:
        print('review' , trainingData['rating'][review])
        if int(trainingData['rating'][review]) == -1:
            neighbourReviewClassNegative+=1
        elif int(trainingData['rating'][review]) == 1:
            neighbourReviewClassPositive+=1
    print(neighbourReviewClassNegative)
    print(neighbourReviewClassPositive)
    if neighbourReviewClassNegative > neighbourReviewClassPositive:
        f.write('-1\n')
        count+=1
    else:
        f.write('+1\n')
        count+=1

print("count : ",count)

In [ ]:
with open("data/test_out.dat.txt", "r") as fh:
    linesOfFormat = fh.readlines()
print(len(linesOfFormat))